<a href="https://www.kaggle.com/tunguz/mnist-xgboost-baseline?scriptVersionId=88491674" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')
sample_submission = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
col = ['pixel%d'%i for i in range(784)]

In [3]:
X_train, X_val, Y_train, Y_val = train_test_split(train[col], train['label'], test_size=0.1)
dtrain = xgb.DMatrix(X_train, label=Y_train)
dval = xgb.DMatrix(X_val, label=Y_val)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [4]:
%%time

watchlist = [(dval, 'eval'), (dtrain, 'train')]

xgb_params = {
    "objective" : "multi:softprob",
    "eval_metric" : "mlogloss",
    "num_class" : 10,
    "max_depth" : 12,
    "eta" : 0.05,
    "subsample" : 0.9,
    "colsample_bytree" : 0.9,
}

xgb_clf = xgb.train(xgb_params, dtrain, 20, 
                    watchlist, 
                    early_stopping_rounds=20, 
                    verbose_eval=20)

[0]	eval-mlogloss:2.10783	train-mlogloss:2.09461
Multiple eval metrics have been passed: 'train-mlogloss' will be used for early stopping.

Will train until train-mlogloss hasn't improved in 20 rounds.
[19]	eval-mlogloss:0.801269	train-mlogloss:0.691162
CPU times: user 12min 40s, sys: 955 ms, total: 12min 41s
Wall time: 3min 14s


In [5]:
res_prob = xgb_clf.predict(xgb.DMatrix(test[col]))


In [6]:
res_prob

array([[0.05646329, 0.04924374, 0.54392123, ..., 0.05127759, 0.04990071,
        0.04962586],
       [0.5504687 , 0.04819364, 0.04898945, ..., 0.04930362, 0.04823476,
        0.04864162],
       [0.06335194, 0.06360564, 0.09620033, ..., 0.06342344, 0.06872007,
        0.365875  ],
       ...,
       [0.04903596, 0.04905185, 0.04964426, ..., 0.04982067, 0.05038926,
        0.04978778],
       [0.05165002, 0.0514019 , 0.05201588, ..., 0.05172733, 0.05346261,
        0.5264819 ],
       [0.04794975, 0.04794277, 0.550302  , ..., 0.04905324, 0.0599061 ,
        0.04930035]], dtype=float32)

In [7]:
np.argmax(res_prob, axis=1)

array([2, 0, 9, ..., 3, 9, 2])

In [8]:
val_preds = np.argmax(xgb_clf.predict(dval), axis=1)

In [9]:
val_preds

array([1, 1, 8, ..., 0, 0, 9])

In [10]:
accuracy_score(Y_val, val_preds)

0.9569047619047619

In [11]:
res = np.argmax(res_prob, axis=1)

In [12]:
res.astype('int')

array([2, 0, 9, ..., 3, 9, 2])

In [13]:
sample_submission['Label'] = res.astype('int')
sample_submission.to_csv('submission.csv', index=False)
np.save('res_prob', res_prob)